In [ ]:
%%capture
!pip install langchain
!pip install tiktoken

In [ ]:
import json

import numpy as np
import pandas as pd
import requests
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from tqdm import tqdm
from transformers import AutoTokenizer

from langchain.text_splitter import TokenTextSplitter

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("microsoft/phi-2", trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token

victor dataset links:

https://drive.google.com/drive/folders/1Mp2f3FbND0TgVPxQrlbi780YD5fkLkEc

In [ ]:
def extract_page_bodies_as_text_array(csv_file, text_array, chunksize = 10000):
  """
  Extracts the "body" content for each page from a CSV file and returns them as a text_array,
  using tqdm for progress visualization.

  Args:
      csv_file (str): Path to the CSV file.

  Returns:
      list: A list of dictionaries, where each dictionary has the following key:
          - "text": The "body" content for a page.
  """

  # Read the CSV file into a DataFrame

  df = pd.read_csv(csv_file)

  # Use tqdm for progress bar
  for index, row in tqdm(df.iterrows(), total=len(df), desc="Extracting page bodies"):
      # Extract document information
      page_body = row["body"]

      # Add page body content to the text_array
      text_array.append({"text": page_body})

  print(extracted_bodies[0])

In [ ]:
def get_docs(url, objects):

  # Make an HTTP GET request to the URL
  response = requests.get(url)

  # Check if the request was successful
  if response.status_code == 200:
      # Load each JSON object into the list
      for line in response.iter_lines():
          json_object = json.loads(line)
          objects.append(json_object)

      # Now json_objects is a list of all the JSON objects in the file
      # For example, you can print the first JSON object
      print(objects[0])
  else:
      print("Failed to retrieve JSON data from the URL")

In [ ]:
# Specify the URL of the JSON file
acordaos_relatorio_url = "https://dadosabertos.c3sl.ufpr.br/acordaos/json/AcordaosRelatorios.json"
acordaos_votos_url = "https://dadosabertos.c3sl.ufpr.br/acordaos/json/AcordaosVotos.json"
#victor_dataset_path = to gain acess to the victor dataset you need to ask trough the link: 

# Create an empty list to store the JSON objects
relatorios = []
votos = []
extracted_bodies = []

get_docs(acordaos_relatorio_url,relatorios)
get_docs(acordaos_votos_url,votos)
#extract_page_bodies_as_text_array(victor_dataset_path,extracted_bodies)

In [ ]:
text_array = [{"text":line["texto"]} for line in relatorios + votos]
text_array.extend(extracted_bodies)
len(text_array)

#### Deduplication

In [ ]:
def deduplicate_exact(text_array):
    """
    Deduplicates text by finding exact matches efficiently.

    Args:
        text_array (list): A list of dictionaries, each containing a "text" key with the textual content.

    Returns:
        list: A deduplicated list of dictionaries, removing exact duplicates.
    """

    unique_texts = []
    seen_texts = set()  # Use a set for efficient membership checks

    for item in tqdm(text_array, desc="Deduplication in progress"):  # Progress bar
        text = item["text"].lower()  # Lowercase for case-insensitive comparison
        if text not in seen_texts:  # Check for membership in the set
            unique_texts.append(item)
            seen_texts.add(text)

    return unique_texts

In [ ]:
text_array = deduplicate_exact(text_array)
len(text_array)

#### Counting tokens

In [ ]:
def calculate_total_tokens(dataset):
    # Load the tokenizer

    total_tokens = 0
    for text in tqdm(dataset, desc="Tokenizing"):
        # Tokenize the text and calculate the number of tokens
        tokens = tokenizer(text["text"], truncation=True, padding=True)
        total_tokens += len(tokens["input_ids"])  # Count tokens using the length of input_ids

    return total_tokens

In [ ]:
total_tokens = calculate_total_tokens(text_array)
print(f"Total number of tokens: {total_tokens}")

#### Saving

In [ ]:
json_file_path = "/content/drive/MyDrive/Projetos/deepL/data/law_dataset_raw.json"


# Save the array to the JSON file
with open(json_file_path, "w") as json_file:
    json.dump(text_array, json_file)